<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2>Submission 1:<br/>Creation of initial data frame.</h2>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib

In [2]:
# Load the data set for parsing with BeautifulSoup
data_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp = urllib.request.urlopen(data_link)

soup = BeautifulSoup(resp.read())
table = soup.table
rows = table.find_all('tr')

<h3>
    Assumptions:
    <li>The zipcode has no hyperlinks and does not need extra processing.</li>
    <br/>
    Process the loaded data and perform the following actions:
    <li>If the row does not contain 3 items, drop the row.</li>
    <li>If a column contains a hyperlink, get the text from the hyperlink.</li>
    <li>If a column does not contain a hyperlink, use the text from that column.</li>
    <li>Remove all trailing whitespace from neighborhoods.</li>
    <li>If the row boroughs is 'Not assigned', drop the row.</li>
    <li>If the row neighborhood is 'Not assigned', use the boroughs name as the neighborhood name.</li>
</h3>

In [3]:
remapped_data = []
for row in rows:
    cols = row.find_all('td')
    if len(cols) == 3:        
        zipcode = cols[0].string

        if cols[1].find('a'):
            borough = cols[1].a.string
        else:
            borough = cols[1].string

        # Skip boroughs with name 'Not assigned'
        if borough == 'Not assigned':
            continue
            
        if cols[2].find('a'):
            neighborhood = cols[2].a.string.rstrip()
        else:
            neighborhood = cols[2].string.rstrip()

        if neighborhood == 'Not assigned':
            neighborhood = borough

        remapped_data.append([zipcode, borough, neighborhood])
        
# print(remapped_data)

<h3>Since multiple neighborhoods can exist in the same zip code, a single zipcode entry needs to aggregate all of the child neighborhoods (separated by commas).  It is assumed that a zipcode will only cover one borough.</h3>

In [4]:
zipcode_map = {}

for entry in remapped_data:
    zipcode = entry[0]
#     print("{} {}".format(entry, zipcode))
    if zipcode not in zipcode_map:
        # add borough and neighborhood to the new entry
        zipcode_map[zipcode] = [entry[1], entry[2]]
    else:
        # append this entry neighborhood to the existing record's neighborhood
        existing_entry = zipcode_map[zipcode]
        existing_entry[1] += ", "
        existing_entry[1] += entry[2]

# print(duplicate_zipcode_map)

In [5]:
# load the dataframe with the contents of the dictionary
df = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'])

i = 0
for entry in zipcode_map:
    zipcode_data = zipcode_map[entry]
    df.loc[i] = [entry, zipcode_data[0], zipcode_data[1]]
    i += 1
    
# print(df)

In [6]:
print(df.shape)

(103, 3)


<h2>Submission 2:<br/>Assign location information.</h2>

<h2>Submission 3:<br/>Visualization of Toronto data.</h2>